# Advent of Code

## 2018-012-015
## 2018 015

https://adventofcode.com/2018/day/15

In [1]:
from collections import deque

def parse_input(file_path):
    """
    Parse the input map and initialize the state of the battlefield.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    grid = []
    units = []

    for y, line in enumerate(lines):
        grid_row = []
        for x, char in enumerate(line.strip()):
            if char in "GE":  # Goblins and Elves
                units.append({
                    'type': char,
                    'hp': 200,
                    'attack': 3,
                    'x': x,
                    'y': y
                })
                grid_row.append('.')
            else:
                grid_row.append(char)
        grid.append(grid_row)

    return grid, units

def reading_order(units):
    """
    Sort units in reading order (top-to-bottom, left-to-right).
    """
    return sorted(units, key=lambda u: (u['y'], u['x']))

def bfs(grid, start, targets):
    """
    Perform a BFS to find the shortest path to any of the targets.
    """
    queue = deque([(start['x'], start['y'], 0)])
    visited = set((start['x'], start['y']))
    paths = []

    while queue:
        x, y, dist = queue.popleft()
        if (x, y) in targets:
            paths.append((dist, y, x))  # Sort by distance, then reading order

        # Explore neighbors in reading order
        for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
            nx, ny = x + dx, y + dy
            if (nx, ny) not in visited and grid[ny][nx] == '.':
                visited.add((nx, ny))
                queue.append((nx, ny, dist + 1))

    return min(paths) if paths else None

def combat_round(grid, units):
    """
    Perform one round of combat. Returns True if combat continues, False if it ends.
    """
    units = reading_order(units)
    for unit in units:
        if unit['hp'] <= 0:
            continue  # Skip dead units

        # Identify targets
        enemies = [u for u in units if u['type'] != unit['type'] and u['hp'] > 0]
        if not enemies:
            return False  # Combat ends

        # Move if not in range of an enemy
        in_range = set()
        for enemy in enemies:
            for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
                nx, ny = enemy['x'] + dx, enemy['y'] + dy
                if grid[ny][nx] == '.':
                    in_range.add((nx, ny))

        if (unit['x'], unit['y']) not in in_range:
            # Find the shortest path to any in-range square
            target = bfs(grid, unit, in_range)
            if target:
                _, ny, nx = target
                grid[unit['y']][unit['x']] = '.'
                unit['x'], unit['y'] = nx, ny
                grid[ny][nx] = unit['type']

        # Attack
        adjacent = []
        for dx, dy in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
            nx, ny = unit['x'] + dx, unit['y'] + dy
            for enemy in enemies:
                if enemy['x'] == nx and enemy['y'] == ny:
                    adjacent.append(enemy)

        if adjacent:
            target = min(adjacent, key=lambda e: (e['hp'], e['y'], e['x']))
            target['hp'] -= unit['attack']
            if target['hp'] <= 0:
                grid[target['y']][target['x']] = '.'

    return True

def calculate_outcome(grid, units):
    """
    Run the simulation and calculate the outcome of the combat.
    """
    rounds = 0
    while combat_round(grid, units):
        rounds += 1

    remaining_hp = sum(u['hp'] for u in units if u['hp'] > 0)
    return rounds * remaining_hp

# Load input and parse the battlefield
file_path = 'input.txt'
grid, units = parse_input(file_path)

# Simulate the combat and calculate the outcome
outcome = calculate_outcome(grid, units)
outcome

143336